# Location stitching script

>This script calls particleLocating.locationStitch to stitch
> particle locations obtained from decon particle locating.
> The files that are read in a csv files that outputted from
> particle locating on ODSY and are indexed by hashValue.

## Steps
1. instantiate a ParticleStitch class (and by inheritance a dpl instance)
- for a given timeStep and material, generate a list of all hashValues
- instatiate locationStitch.ParticleStitch class
- for the given hashValues, stitch their locations
- filter the resulting dataFrame by keepBool
- save **all** columns of the filtered dataframe to hdf5 file
- stream the results for all time steps
- after all time steps are complete, run particle linking using tp.link_df and stream from hdf5 data store
- carry out the above steps for the other material (sed and gel)
- consider archiving this hdf5 store, and splitting into smaller stores by eliminating some columns
> this single data store is going to be quite large
> and many of the columns are going to be accessed rarely if ever
> Additionally, downstream analysis is going to rapdily multiply
> the number of columns.
> I would regularly access stitched and tracked position data (three columns, plus frame number)
> occssionally access three additional column of location uncertainty
> and rarely access other metric like mass or signal in order to do quality control on particle
> positions (eg is this high strain cluster really high strain or a fluorescent chunk in which
> the apparent peak locations are fluctuating.


In [1]:
from importlib import reload
import particleLocating
from particleLocating.locationStitch import ParticleStitch
from importlib import reload
reload(particleLocating)

yamlPath = '/Users/zsolt/Colloid_git/TractionRheoscopy/metaDataYAML/tfrGel10212018A_shearRun10292018f_metaData.yaml'
inst = ParticleStitch(yamlPath)

In [2]:
timeRange = range(inst.dpl.metaData['hashDimensions']['timeSteps']+1)
timeRange = range(0,1)
for t in timeRange:
    hvList_sed, hvList_gel = inst.dpl.time2HVList(t,'sed'), inst.dpl.time2HVList(t,'gel')
    inst.stitch(hvList_sed)
    loc = inst.locations
    keep = loc[loc['keepBool']]
    overlap = loc[~loc['keepBool']]
    inst.df2h5(keep,'sed', '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/locations_stitch')

/Users/zsolt/.conda/envs/TractionRheoscopy/lib/python3.7/site-packages/numba/core/typed_passes.py:314: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../particleLocating/locationStitch.py", line 189:
    @numba.jit(nopython=True, nogil=True, parallel=True)
    def minError_flagDouble(pairList, errorList):
    ^

  state.func_ir.loc))
/Users/zsolt/.conda/envs/TractionRheoscopy/lib/python3.7/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'Frame_0.0'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


TypeError: object of type 'int' has no len()

In [4]:
type(  keep)


pandas.core.frame.DataFrame

In [ ]:
import trackpy as tp
with tp.PandasHDFStore('/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/locations_stitch/tfrGel10212018A_shearRun10292018f_sed_stitched.h5') as s:
    s.put(keep)

